# Rotator Streanous

This notebook was prepared to do the testing of the rotator in coordination with the pointing kernel and the camera cable wrap.
The goal is to send command to the rotator to go to 90 at azimuth = 0, wait for a minute and then send the rotator to -90 at azimuth = 180

In [1]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

from datetime import datetime

In [2]:
now = datetime.now()
test_message = "Rotator integration test, Streneous motion"
print(test_message, now)

Rotator integration test, Streneous motion 2019-12-12 15:04:22.415255


In [3]:
d = salobj.Domain()

In [4]:
def parallactic_angle(location, lst, target):
    """
    Calculate the parallactic angle.
    Parameters
    ----------
    time : `~astropy.time.Time`
        Observation time.
    target : `~astroplan.FixedTarget` or `~astropy.coordinates.SkyCoord` or list
        Target celestial object(s).
    grid_times_targets: bool
        If True, the target object will have extra dimensions packed onto the end,
        so that calculations with M targets and N times will return an (M, N)
        shaped result. Otherwise, we rely on broadcasting the shapes together
        using standard numpy rules.
    Returns
    -------
    `~astropy.coordinates.Angle`
        Parallactic angle.
    Notes
    -----
    The parallactic angle is the angle between the great circle that
    intersects a celestial object and the zenith, and the object's hour
    circle [1]_.
    .. [1] https://en.wikipedia.org/wiki/Parallactic_angle
    """
    # Eqn (14.1) of Meeus' Astronomical Algorithms
    H = (lst - target.ra).radian
    q = np.arctan2(np.sin(H),
                   (np.tan(location.lat.radian) *
                    np.cos(target.dec.radian) -
                    np.sin(target.dec.radian)*np.cos(H)))*u.rad
    return Angle(q)

In [5]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [6]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")
mtm = salobj.Remote(d, "MTMount")

In [7]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task,
                     mtm.start_task)

RemoteTelemetry(Rotator, 0, Motors) falling behind; read 13 messages


[None, None, None, None]

RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 14 messages


In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await mtm.cmd_enterControl.start(timeout=10.)
await mtm.cmd_start.start(timeout=10.)
await mtm.cmd_enable.start(timeout=10.)

# Start at 0

In [8]:
now = datetime.now()
print("Start at 0", now)

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)
    
print("Waiting 30s")
await asyncio.sleep(30.)

RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages


Start at 0 2019-12-12 15:05:45.860595


RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 15 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 75 messages


Time error=-57.01 sec


RemoteTelemetry(MTPtg, 0, currentTimesToLimits) falling behind; read 15 messages


Alt: 45.0 deg
Az: 0.0 deg
RotTel: 0.0 deg
ParaAng: 179.7314801607311 deg
0d00m00s
Waiting 30s


# Start Test

In [15]:
now = datetime.now()
print("Start Test", now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message}")

Start Test 2019-12-12 15:21:37.055596


False

# Go to 90 and then -90

In [23]:
now = datetime.now()
print("Azimuth = 0 and rot_tel= 89 deg", now)

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(89, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)
    
print("Waiting 35s")
await asyncio.sleep(35.)
    
    
# # ########################################################################################### 
now = datetime.now()    
print("Azimuth = 180 and rot_tel= -89 deg", now)

alt = 45. * u.deg
az = 180. * u.deg
rot_tel = Angle(-89, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print(rot_pa)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)
    
print("Waiting 65s")
await asyncio.sleep(65.)
    
    
# ########################################################################################### 
       

RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages


Azimuth = 0 and rot_tel= 89 deg 2019-12-12 15:26:24.925059


RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 36 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages


Time error=-56.99 sec
Alt: 45.0 deg
Az: 0.0 deg
RotTel: 89.0 deg
ParaAng: 179.732871650721 deg
88d43m58.3379s
Waiting 35s
Azimuth = 180 and rot_tel= -89 deg 2019-12-12 15:27:00.689614
Time error=-37.00 sec
Alt: 45.0 deg
Az: 180.0 deg
RotTel: -89.0 deg
ParaAng: 0.7256378475185241 deg
-268d16m27.7037s
Waiting 65s


# Back to park position

In [ ]:
now = datetime.now()
print("Move to park position", now)

alt = 60. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)    
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 30s")
await asyncio.sleep(30.)



In [ ]:
now = datetime.now()
print("Stop tracking",now)
#     #we need to stop the pointing commands before stopping the rotator. This will be integrated in the pointing code
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)

# Test Complete

In [ ]:
now = datetime.now()
print("Test Complete",now)

script.evt_logMessage.set_put(level=logging.INFO+1,
                          message=f"END - {test_message}")

# Additional Error Handling Commands

# Bring the Rotator and CCW back to enabled state

In [19]:
await mtm.cmd_clearerror.start(timeout=30)

RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages


Wait for override off

In [20]:
await rot.cmd_clearError.start(timeout=30)

RemoteEvent(MTMount, 0, mountState) falling behind; read 34 messages


In [21]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

[<State.OFFLINE: 4>,
 <State.STANDBY: 5>,
 <State.DISABLED: 1>,
 <State.ENABLED: 2>]

In [22]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

# Move to sync position with CCW

In [ ]:
print("Move to X.X deg starting position")

await rot.cmd_positionSet.set_start(angle=19.04,timeout=30.)

await rot.cmd_move.start(timeout=30.)

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await rot.cmd_stop.start(timeout=10.)

In [ ]:
await mtm.cmd_clearerror.start(timeout=10)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10)
await mtm.cmd_stop.start(timeout=10)

In [ ]:
await mtm.cmd_stop.start(timeout=10)

Bring CCW Down

In [ ]:
await mtm.cmd_disable.start(timeout=30.)
await mtm.cmd_standby.start(timeout=30.)
await mtm.cmd_exitControl.start(timeout=30.)

Wait for override off

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

In [16]:
await mtptg.cmd_stopTracking.start(timeout=30)

RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 56 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=2089407656, ack=<SalRetCode.CMD_FAILED: -302>, error=6612, result='Rejected : command not allowed in current state')

In [18]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)

RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages


[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]